In [1]:
import pandas as pd
import numpy as np
import seaborn as sp
import matplotlib as mplot
from collections import defaultdict
from scipy import stats

In [2]:
weeksalesdb = pd.read_csv('train.csv')
stores = pd.read_csv('stores.csv' , dtype = {'Store': int})
features = pd.read_csv('features.csv')

In [3]:
features.drop(columns = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], inplace = True)

In [4]:
#converts weeksalesdb rows into dict and merging departments
agg_sales = defaultdict(int)
for ind, sale in weeksalesdb.iterrows():
    agg_sales[str(sale['Store'])+'.'+sale['Date']] += sale['Weekly_Sales']

In [5]:
#converts back into db with updated sales value
storesales = pd.DataFrame(columns=['Store','Date','Weekly_Sales'])

for key, value in agg_sales.items():
    store, date = key.split('.')
    newrow = [store, date, value]
    storesales.loc[len(storesales)] = newrow

In [6]:
#function to convert date to integer
def convert_date(date):
    date = date.strip()
    date = date.replace('-','')
    date = date.strip()
    return int(date)

#convert date columns
storesales['Date'] = storesales['Date'].apply(convert_date)
features['Date'] = features['Date'].apply(convert_date)

In [7]:
#convert store column to int (was str)
storesales['Store'] = pd.to_numeric(storesales['Store'])
features.head(10)

,Store,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday
0,1,20100205,42.31,2.572,211.096358,8.106,False
1,1,20100212,38.51,2.548,211.242170,8.106,True
2,1,20100219,39.93,2.514,211.289143,8.106,False
3,1,20100226,46.63,2.561,211.319643,8.106,False
4,1,20100305,46.50,2.625,211.350143,8.106,False
5,1,20100312,57.79,2.667,211.380643,8.106,False
6,1,20100319,54.58,2.720,211.215635,8.106,False
7,1,20100326,51.45,2.732,211.018042,8.106,False
8,1,20100402,62.27,2.719,210.820450,7.808,False
9,1,20100409,65.86,2.770,210.622857,7.808,False


In [8]:
#exported csv files
storesales.to_csv('storesales.csv')
features.to_csv('features2.csv')

In [9]:
#merged two dataframes together on the store and date column
merged = pd.merge(storesales, features, on =['Store', 'Date'], how = 'outer')

In [10]:
#removed rows where unemployment data was empty
merged.dropna(subset = ['Unemployment'], inplace = True)

In [11]:
#export cleaned features
merged.to_csv('features_clean.csv', index = False)

In [12]:
merged.head(10)

,Store,Date,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday
0,1,20100205,1643690.90,42.31,2.572,211.096358,8.106,False
1,1,20100212,1641957.44,38.51,2.548,211.242170,8.106,True
2,1,20100219,1611968.17,39.93,2.514,211.289143,8.106,False
3,1,20100226,1409727.59,46.63,2.561,211.319643,8.106,False
4,1,20100305,1554806.68,46.50,2.625,211.350143,8.106,False
5,1,20100312,1439541.59,57.79,2.667,211.380643,8.106,False
6,1,20100319,1472515.79,54.58,2.720,211.215635,8.106,False
7,1,20100326,1404429.92,51.45,2.732,211.018042,8.106,False
8,1,20100402,1594968.28,62.27,2.719,210.820450,7.808,False
9,1,20100409,1545418.53,65.86,2.770,210.622857,7.808,False


In [13]:
#checked for outliers in the weekly sales column, most of these are on or near holidays.
outliers = merged[merged['Weekly_Sales'] > merged['Weekly_Sales'].mean() + 3 * merged['Weekly_Sales'].std()]
outliers

,Store,Date,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday
189,2,20101224,3436007.68,49.97,2.886,211.064660,8.163,False
241,2,20111223,3224369.80,46.66,3.112,218.999550,7.441,False
471,4,20101126,2789469.45,48.08,2.752,126.669267,7.127,True
475,4,20101224,3526713.39,43.21,2.887,126.983581,7.127,False
523,4,20111125,3004702.33,47.96,3.225,129.836400,5.143,True
526,4,20111216,2771397.17,36.44,3.149,129.898065,5.143,False
527,4,20111223,3676388.98,35.92,3.103,129.984548,5.143,False
1329,10,20101126,2939946.38,55.33,3.162,126.669267,9.003,True
1332,10,20101217,2811646.85,59.15,3.125,126.879484,9.003,False
1333,10,20101224,3749057.69,57.06,3.236,126.983581,9.003,False
